In [40]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.optimizers import Adam

import joblib

mpl.style.use('default')
plt.rc('text',usetex = True)
plt.rc('font', family='serif',size = 12)

In [41]:
def plot_Wm(y_train,y_pred_train,y_test,y_pred_test, Method):
    Aux = np.linspace(10,60)
    plt.figure()
    plt.plot(Aux,0.9*Aux,'k--')
    plt.plot(Aux,1.1*Aux,'k--')
    plt.plot(y_train,y_pred_train,'o',color = '#1f77b4', mfc='none', label = 'Training Set')
    plt.plot(y_test,y_pred_test,'x' ,color = '#2ca02c', mfc='none',label = 'Test Set',)
    plt.text(23, 17, '$- 10 \%$', fontsize=12)
    plt.text(19, 28, '$+ 10 \%$', fontsize=12)
    plt.legend()
    plt.ylabel(r' $\dot{W}_\mathrm{AMR,Mag}$ [W] - ' + Method )
    plt.xlabel(r' $\dot{W}_\mathrm{AMR,Mag}$ [W] - Numerical Solution')
    plt.grid(linestyle='dotted')
    plt.savefig('Plots/Wm - '+Method +'.png', format = 'png', bbox_inches='tight',) 

In [42]:
def plot_Qc(y_train,y_pred_train,y_test,y_pred_test, Method):
    Aux = np.linspace(10,300)
    plt.figure()
    plt.plot(Aux,0.9*Aux,'k--')
    plt.plot(Aux,1.1*Aux,'k--')
    plt.plot(y_train,y_pred_train,'o',color = '#1f77b4', mfc='none', label = 'Training Set')
    plt.plot(y_test,y_pred_test,'x' ,color = '#2ca02c', mfc='none',label = 'Test Set')
    plt.text(140, 180, '$+ 10 \%$', fontsize=12)
    plt.text(150, 120, '$- 10 \%$', fontsize=12)
    plt.ylabel(r' $\dot{Q}_\mathrm{C}$ [W] - ' + Method )
    plt.xlabel(r' $\dot{Q}_\mathrm{C}$ [W] - Target')
    plt.legend()
    plt.grid(linestyle='dotted')
    plt.savefig('Plots/Qc - '+Method +'.png', format = 'png', bbox_inches='tight',) 

### Reading the Input Parameteres 

In [43]:
Inputs = pd.read_excel('Data/Data_AMR.xlsx',index_col = 0) # Input Data

# Train-Test Split

In [44]:
X = Inputs[['W','H','f','mf','B','L','Th','Tc']]
y = Inputs[['Qc','Wm']]
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = 42)  

In [45]:
sc = StandardScaler()
x_train = sc.fit_transform(X_train)
x_test = sc.transform(X_test)

In [46]:
def build_model2(hp):
    model = tf.keras.Sequential()
    model.add(BatchNormalization())
    for i in range(hp.Int('layers', 0, 3)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), 10, 10100, step=100),
                                        activation=hp.Choice('act_' + str(i), ['relu','sigmoid','selu'])))
        model.add(BatchNormalization())
    model.add(tf.keras.layers.Dense(1, activation='selu'))
    model.compile(optimizer=Adam(),
                  loss="mean_squared_error", 
                  metrics=['accuracy'])
    return model

class MyTuner(kt.Hyperband):
    def run_trial(self, trial, *args, **kwargs):
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 20, 200, step=10)
        super(MyTuner, self).run_trial(trial, *args, **kwargs)

In [47]:
tuner = MyTuner(build_model2,
                objective = 'val_loss',
                max_epochs = 50,
                directory='KT_Dir',
                project_name='KT_Hyperband',
                executions_per_trial = 1,   
)

In [48]:
tuner.search(x_train,
             y_train.values,
             epochs=50, 
             verbose=1,
             validation_split=0.2)

Trial 41 Complete [00h 01m 26s]
val_loss: 8823.39453125

Best val_loss So Far: 6961.7236328125
Total elapsed time: 00h 13m 45s

Search: Running Trial #42

Hyperparameter    |Value             |Best Value So Far 
layers            |2                 |2                 
batch_size        |170               |90                
units_0           |9510              |6410              
act_0             |selu              |sigmoid           
units_1           |3910              |2010              
act_1             |relu              |selu              
tuner/epochs      |6                 |6                 
tuner/initial_e...|2                 |2                 
tuner/bracket     |3                 |3                 
tuner/round       |1                 |1                 
tuner/trial_id    |cbf6f1ee0b77626...|2c129d1015d8fa4...

Epoch 3/6
2/2 [==============================] - 7s 4s/step - loss: 12536.4736 - accuracy: 0.0000e+00 - val_loss: 13793.8926 - val_accuracy: 0.0000e+00
Epoch 4/

KeyboardInterrupt: 

In [ ]:
best_model = tuner2.get_best_models()[0]
best_model.build(x_train.shape)

In [ ]:
best_model.fit(x_train,
               y_train.values, 
               epochs=100,
               batch_size=100
)

In [ ]:
B_pred = best_model.predict(x_train)
B_pred_test = best_model.predict(x_test)

In [ ]:
best_model.save('Numerical.h5')

In [ ]:
r2_score(y_test,B_pred_test)

In [ ]:
B_error = np.arange(50,300,0.1)
plt.plot(B_error,0.9*B_error,'k--')
plt.plot(B_error,1.1*B_error,'k--')


#plt.plot(B_error,1*B_error,'r--')
plt.plot(y_train,B_pred,'o',color = '#1f77b4', mfc='none', label = 'Training Set')
plt.plot(y_test,B_pred_test,'x',color = '#2ca02c', mfc='none',label = 'Test Set',alpha = 1)


plt.text(140, 180, '$+ 10 \%$', fontsize=12)
plt.text(150, 120, '$- 10 \%$', fontsize=12)
plt.grid(linestyle = 'dashed')
plt.legend()
fig = matplotlib.pyplot.gcf()
#fig.set_size_inches(14, 8)

plt.xlabel('$\dot{Q}_\mathrm{C}$[W] - Target', fontsize = 12)
plt.ylabel('$\dot{Q}_\mathrm{C}$[W] - Machine Learning Prediction', fontsize = 12)
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

#plt.text(120, 100, '$-4 \%$', fontsize=12)
#plt.text(120, 160,  '$+4 \%$', fontsize=12)
#plt.xlim([60,300])
#print('score (acurácia) = ', model.score(X_train,B_train))

plt.savefig('Qc_NN_Exp.pdf',format = 'pdf', dpi=500,bbox_inches='tight')